In [3]:
import json 
import phaselocker as pl
import numpy as np 
from bokeh.io import output_notebook, show
import phaselocker.geometry as geom
import phaselocker.sampling as mc 


In [4]:
#Load calculated LiAl data
with open('data/calculated_configs.json','r') as f:
    calc_data = json.load(f)
calc_comp = np.array(calc_data['comp'])
calc_corr = np.array(calc_data['corr'])
calc_ef = np.array(calc_data['formation_energy'])

#Load uncalculated data
with open("data/uncalculated_configs.json",'r') as f:
    uncalc_data = json.load(f)
uncalc_comp = np.array(uncalc_data['comp'])
uncalc_corr = np.array(uncalc_data['corr'])

#Combine datasets to make an over-enumerated dataset
all_comp = np.vstack([calc_comp, uncalc_comp])
all_corr = np.vstack([calc_corr, uncalc_corr])


In [5]:
#Find the observed ground state incides
observed_hull = geom.full_hull(compositions=calc_comp, energies=calc_ef)
observed_gs_indices, simplices = geom.lower_hull(observed_hull)

#Enforce the observed ground states
imposed_vertices = observed_gs_indices
imposed_simplices = observed_hull.simplices[simplices]
print(observed_gs_indices)

[  0   1  15  28  49  83 267 441]


In [6]:
#Make a gradient descent function to find the cone

def find_cone(starting_point):
    current_point = starting_point
    for step in range(5000):
        predicted_hull = geom.full_hull(all_comp, all_corr @ current_point)
        _, predicted_simplices = geom.lower_hull(predicted_hull)
        predicted_simplices = predicted_hull.simplices[predicted_simplices]
        neg_grad = geom.orderparam_gradient(
            imposed_gs_corr=calc_corr,
            imposed_gs_comp=calc_comp,
            imposed_gs_simplices=imposed_simplices,
            overenum_corr=all_corr,
            overenum_comp = all_comp,
            predicted_simplices=predicted_simplices,
        )
        gradmag = np.linalg.norm(neg_grad)
        if gradmag < 1e-9:
            return current_point
        current_point += 0.002 * neg_grad / gradmag
    return np.nan

In [7]:
#Use an initial ECI vector to initialize cone search
init_eci = np.random.rand(calc_corr.shape[1])
init_eci = init_eci /np.linalg.norm(init_eci)


In [8]:
#Find the cone using the initial ECI vector and the gradient descent function
final_eci = find_cone(init_eci)

In [10]:
print(final_eci)

[ 0.02545604  0.06273853  0.0344862   0.07651785 -0.01177291  0.03687073
 -0.00503199  0.03195799  0.00895255 -0.00126747 -0.01714049  0.01862584
  0.04808763  0.03794482 -0.00375338  0.05344295  0.04604408 -0.00505729
  0.00752162  0.03782997  0.03724461 -0.00588705  0.06191004 -0.01283004
  0.0139661   0.03602559 -0.01580868 -0.00927631  0.04268356  0.01947948
  0.0037219  -0.00543517  0.03589701  0.0042575  -0.029732    0.03895383
 -0.02290159  0.0447694  -0.01587112  0.00939379 -0.00084605  0.03715306
  0.0034233  -0.0228901  -0.02719235  0.03546656 -0.0134291   0.05328454
 -0.00619664  0.00450212  0.03921618  0.04867293 -0.02093499  0.01196343
  0.04378167  0.01894988  0.0272305   0.02167786 -0.00792943  0.0196851
  0.02899107  0.05173841  0.0078989   0.05713925 -0.00727493 -0.02009464
 -0.03243572  0.03647961 -0.0065432  -0.04466923  0.0058666  -0.0154391
  0.04437028  0.02361536  0.01823458  0.03274234  0.02946482 -0.00520436
  0.02063087  0.04179702  0.01919606 -0.01181463 -0.0